# Model Fit in Linear Regression - Lab

## Introduction
In this lab, you'll learn how to evaluate your model results and you'll learn how to select the appropriate features using stepwise selection.

## Objectives
You will be able to:
* Use stepwise selection methods to determine the most important features for a model
* Use recursive feature elimination to determine the most important features for a model

## The Ames Housing Data once more

In [13]:
import pandas as pd
import numpy as np

ames = pd.read_csv('ames.csv')

continuous = ['LotArea', '1stFlrSF', 'GrLivArea', 'SalePrice']
categoricals = ['BldgType', 'KitchenQual', 'SaleType', 'MSZoning', 'Street', 'Neighborhood']

ames_cont = ames[continuous]

# log features
log_names = [f'{column}_log' for column in ames_cont.columns]

ames_log = np.log(ames_cont)
ames_log.columns = log_names

# normalize (subract mean and divide by std)

def normalize(feature):
    return (feature - feature.mean()) / feature.std()

ames_log_norm = ames_log.apply(normalize)

# one hot encode categoricals
ames_ohe = pd.get_dummies(ames[categoricals], prefix=categoricals, drop_first=True)

preprocessed = pd.concat([ames_log_norm, ames_ohe], axis=1)

## Perform stepwise selection

The function for stepwise selection is copied below. Use this provided function on your preprocessed Ames Housing data.

In [19]:
preprocessed.columns

Index(['LotArea_log', '1stFlrSF_log', 'GrLivArea_log', 'SalePrice_log',
       'BldgType_2fmCon', 'BldgType_Duplex', 'BldgType_Twnhs',
       'BldgType_TwnhsE', 'KitchenQual_Fa', 'KitchenQual_Gd', 'KitchenQual_TA',
       'SaleType_CWD', 'SaleType_Con', 'SaleType_ConLD', 'SaleType_ConLI',
       'SaleType_ConLw', 'SaleType_New', 'SaleType_Oth', 'SaleType_WD',
       'MSZoning_FV', 'MSZoning_RH', 'MSZoning_RL', 'MSZoning_RM',
       'Street_Pave', 'Neighborhood_Blueste', 'Neighborhood_BrDale',
       'Neighborhood_BrkSide', 'Neighborhood_ClearCr', 'Neighborhood_CollgCr',
       'Neighborhood_Crawfor', 'Neighborhood_Edwards', 'Neighborhood_Gilbert',
       'Neighborhood_IDOTRR', 'Neighborhood_MeadowV', 'Neighborhood_Mitchel',
       'Neighborhood_NAmes', 'Neighborhood_NPkVill', 'Neighborhood_NWAmes',
       'Neighborhood_NoRidge', 'Neighborhood_NridgHt', 'Neighborhood_OldTown',
       'Neighborhood_SWISU', 'Neighborhood_Sawyer', 'Neighborhood_SawyerW',
       'Neighborhood_Somerst', 'Nei

In [33]:
import statsmodels.api as sm

def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in=0.01, 
                       threshold_out = 0.05, 
                       verbose=True):
    """ 
    Perform a forward-backward feature selection 
    based on p-value from statsmodels.api.OLS
    Arguments:
        X - pandas.DataFrame with candidate features
        y - list-like with the target
        initial_list - list of features to start with (column names of X)
        threshold_in - include a feature if its p-value < threshold_in
        threshold_out - exclude a feature if its p-value > threshold_out
        verbose - whether to print the sequence of inclusions and exclusions
    Returns: list of selected features 
    Always set threshold_in < threshold_out to avoid infinite looping.
    See https://en.wikipedia.org/wiki/Stepwise_regression for the details
    """
    included = list(initial_list)
    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded)
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.idxmin()
            included.append(best_feature)
            changed=True
            if verbose:
                print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # backward step
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.idxmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included

In [34]:
X = preprocessed.drop('SalePrice_log', axis=1)
y = preprocessed['SalePrice_log']

result = stepwise_selection(X, y, initial_list=X.columns, verbose = True)
print('resulting features:')
print(result)

<ipython-input-33-470561526870>:27: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_pval = pd.Series(index=excluded)
<ipython-input-33-470561526870>:27: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_pval = pd.Series(index=excluded)
<ipython-input-33-470561526870>:27: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_pval = pd.Series(index=excluded)
<ipython-input-33-470561526870>:27: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_pval = pd.Series(index=excluded)
<ipython-input-33-470561526870>:27: 

Drop Neighborhood_Timber            with p-value 0.966131
Drop SaleType_ConLw                 with p-value 0.93525
Drop Neighborhood_NPkVill           with p-value 0.898951
Drop SaleType_ConLI                 with p-value 0.873264
Drop Neighborhood_Blueste           with p-value 0.867178
Drop SaleType_Oth                   with p-value 0.644009
Drop Neighborhood_Somerst           with p-value 0.381603


<ipython-input-33-470561526870>:27: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_pval = pd.Series(index=excluded)
<ipython-input-33-470561526870>:27: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_pval = pd.Series(index=excluded)
<ipython-input-33-470561526870>:27: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_pval = pd.Series(index=excluded)


Drop Neighborhood_Veenker           with p-value 0.443212
Drop SaleType_CWD                   with p-value 0.298901


<ipython-input-33-470561526870>:27: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_pval = pd.Series(index=excluded)
<ipython-input-33-470561526870>:27: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_pval = pd.Series(index=excluded)


...

Drop BldgType_Twnhs                 with p-value 0.211941
Drop Neighborhood_Crawfor           with p-value 0.145835


<ipython-input-33-470561526870>:27: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_pval = pd.Series(index=excluded)
<ipython-input-33-470561526870>:27: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_pval = pd.Series(index=excluded)


Drop Neighborhood_CollgCr           with p-value 0.183425
Drop Neighborhood_Gilbert           with p-value 0.36153


<ipython-input-33-470561526870>:27: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_pval = pd.Series(index=excluded)
<ipython-input-33-470561526870>:27: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_pval = pd.Series(index=excluded)


resulting features:
['LotArea_log', '1stFlrSF_log', 'GrLivArea_log', 'BldgType_2fmCon', 'BldgType_Duplex', 'KitchenQual_Fa', 'KitchenQual_Gd', 'KitchenQual_TA', 'SaleType_Con', 'SaleType_ConLD', 'SaleType_New', 'SaleType_WD', 'MSZoning_FV', 'MSZoning_RH', 'MSZoning_RL', 'MSZoning_RM', 'Neighborhood_BrDale', 'Neighborhood_BrkSide', 'Neighborhood_ClearCr', 'Neighborhood_Edwards', 'Neighborhood_IDOTRR', 'Neighborhood_MeadowV', 'Neighborhood_Mitchel', 'Neighborhood_NAmes', 'Neighborhood_NWAmes', 'Neighborhood_NoRidge', 'Neighborhood_NridgHt', 'Neighborhood_OldTown', 'Neighborhood_SWISU', 'Neighborhood_Sawyer', 'Neighborhood_SawyerW', 'Neighborhood_StoneBr']


### Build the final model again in Statsmodels

In [35]:
from statsmodels.formula.api import ols

In [93]:
import statsmodels.api as sm
X_fin = X[result]
X_with_intercept = sm.add_constant(X_fin)
model = sm.OLS(y,X_with_intercept).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          SalePrice_log   R-squared:                       0.838
Model:                            OLS   Adj. R-squared:                  0.834
Method:                 Least Squares   F-statistic:                     230.1
Date:                Thu, 27 May 2021   Prob (F-statistic):               0.00
Time:                        12:11:11   Log-Likelihood:                -743.88
No. Observations:                1460   AIC:                             1554.
Df Residuals:                    1427   BIC:                             1728.
Df Model:                          32                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                   -0.3652      0.171     -2.131      0.033      -0.701      -0.029
LotArea_log              0.1169      0.015      7.706      0.000       0.087       0.147
1stFlrSF_log             0.1460      0.015      9.779      0.000       0.117       0.175
GrLivArea_log            0.3707      0.015     24.669      0.000       0.341       0.400
BldgType_2fmCon         -0.1740      0.079     -2.214      0.027      -0.328      -0.020
BldgType_Duplex         -0.4091      0.061     -6.734      0.000      -0.528      -0.290
KitchenQual_Fa          -1.0129      0.087    -11.578      0.000      -1.185      -0.841
KitchenQual_Gd          -0.3959      0.050     -7.957      0.000      -0.494      -0.298
KitchenQual_TA          -0.6840      0.055    -12.550      0.000      -0.791      -0.577
SaleType_Con             0.6419      0.295      2.179      0.030       0.064       1.220
SaleType_ConLD           0.3199      0.150      2.130      0.033       0.025       0.615
SaleType_New             0.2921      0.069      4.250      0.000       0.157       0.427
SaleType_WD              0.1522      0.055      2.780      0.006       0.045       0.260
MSZoning_FV              1.1988      0.167      7.165      0.000       0.871       1.527
MSZoning_RH              0.8711      0.191      4.564      0.000       0.497       1.246
MSZoning_RL              0.9915      0.158      6.278      0.000       0.682       1.301
MSZoning_RM              1.0826      0.149      7.265      0.000       0.790       1.375
Neighborhood_BrDale     -0.4496      0.123     -3.663      0.000      -0.690      -0.209
Neighborhood_BrkSide    -0.5720      0.066     -8.621      0.000      -0.702      -0.442
Neighborhood_ClearCr    -0.1725      0.083     -2.088      0.037      -0.335      -0.010
Neighborhood_Edwards    -0.7053      0.049    -14.510      0.000      -0.801      -0.610
Neighborhood_IDOTRR     -0.8673      0.097     -8.967      0.000      -1.057      -0.678
Neighborhood_MeadowV    -0.6596      0.118     -5.596      0.000      -0.891      -0.428
Neighborhood_Mitchel    -0.2056      0.065     -3.175      0.002      -0.333      -0.079
Neighborhood_NAmes      -0.3854      0.038    -10.094      0.000      -0.460      -0.310
Neighborhood_NWAmes     -0.2156      0.054     -4.001      0.000      -0.321      -0.110
Neighborhood_NoRidge     0.4155      0.070      5.944      0.000       0.278       0.553
Neighborhood_NridgHt     0.3814      0.056      6.758      0.000       0.271       0.492
Neighborhood_OldTown    -0.8523      0.064    -13.283      0.000      -0.978      -0.726
Neighborhood_SWISU      -0.6279      0.089     -7.050      0.000      -0.803      -0.453
Neighborhood_Sawyer     -0.4192      0.055     -7.618      0.000      -0.527      -0.311
Neighborhood_SawyerW    -0.1809      0.058     -3.108    

## Use Feature ranking with recursive feature elimination

Use feature ranking to select the 5 most important features

In [106]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

linreg = LinearRegression()
selector = RFE(linreg, n_features_to_select = 5)
selector = selector.fit(X, y.values.ravel()) # convert y to 1d np array to prevent DataConversionWarning
selector.support_ 

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
        True,  True,  True,  True, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False,  True, False, False, False, False, False, False, False,
       False, False])

Fit the linear regression model again using the 5 selected columns

In [109]:
selected_columns = X.columns[selector.support_ ]
linreg.fit(X[selected_columns],y)

LinearRegression()

Now, predict $\hat y$ using your model. You can use `.predict()` in scikit-learn. 

In [113]:
y_hat = linreg.predict(X[selected_columns])

Now, using the formulas of R-squared and adjusted R-squared below, and your Python/numpy knowledge, compute them and contrast them with the R-squared and adjusted R-squared in your statsmodels output using stepwise selection. Which of the two models would you prefer?

$SS_{residual} = \sum (y - \hat{y})^2 $

$SS_{total} = \sum (y - \bar{y})^2 $

$R^2 = 1- \dfrac{SS_{residual}}{SS_{total}}$

$R^2_{adj}= 1-(1-R^2)\dfrac{n-1}{n-p-1}$

In [125]:
ssr = np.sum((y-y_hat)**2)
sst = np.sum((y-np.mean(y))**2)
r_squared = 1 - (ssr/sst)

adj_r_squared = 1 - ((1-r_squared)* ((len(y)-1)) / (len(y)-X[selected_columns].shape[1]-1))
r_squared, adj_r_squared

# r_squared is 0.239434  
# adjusted_r_squared is 0.236818

(0.23943418177114228, 0.2368187559863113)

## Level up (Optional)

- Perform variable selection using forward selection, using this resource: https://planspace.org/20150423-forward_selection_with_statsmodels/. Note that this time features are added based on the adjusted R-squared!
- Tweak the code in the `stepwise_selection()` function written above to just perform forward selection based on the p-value 

## Summary
Great! You practiced your feature selection skills by applying stepwise selection and recursive feature elimination to the Ames Housing dataset! 